Building file paths for local use

In [1]:
import pandas as pd
import os

# Define the base directory where all your Eurostat files are stored
BASE_DIR = os.path.expanduser(r"~\OneDrive\Desktop\TIL Programming\6020 Group project\Project data_Freight") # change this to your folder path

# Build file paths safely using os.path.join
files = {
    "rail_go_total": os.path.join(BASE_DIR, "rail_go_total__custom_18309054_linear_2_0.csv"),
    "rail_if_line_na": os.path.join(BASE_DIR, "rail_if_line_na__custom_18324484_linear_2_0.csv"),
    "tran_hv_frmod": os.path.join(BASE_DIR, "tran_hv_frmod__custom_18309026_linear_2_0.csv"),
    "ttr00006": os.path.join(BASE_DIR, "ttr00006__custom_18309048_linear_2_0.csv"),
    "rail_go_consgmt": os.path.join(BASE_DIR, "rail_go_consgmt__custom_18308929_linear_2_0.csv"),
    "rail_go_grpgood": os.path.join(BASE_DIR, "rail_go_grpgood__custom_18309135_linear_2_0.csv")}

# EU27 country list (2020 definition)
EU27 = [
    "BE","BG","CZ","DK","DE","EE","IE","EL","ES","FR","HR","IT","CY",
    "LV","LT","LU","HU","MT","NL","AT","PL","PT","RO","SI","SK","FI","SE"]

Cleaning Functions

In [2]:
#Cleaning Geopolitcal data
def clean_geo(df):
    """Remove EU aggregates and keep only EU27 countries."""
    aggregates_pattern = r'EU|EA|EFTA|EWR|EUR|TOT'
    df = df[~df['geo'].astype(str).str.contains(aggregates_pattern, case=False, na=False)]
    df = df[df['geo'].isin(EU27)]
    return df

#Verify if each version is readable
def read_eurostat(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    print(f"Loading: {os.path.basename(path)}")
    return pd.read_csv(path, engine="python")

Loading and cleaning seperate Data sets

In [3]:
# Goods transported by rail (total)--------------------------------------------------
rail_go_total = read_eurostat(files["rail_go_total"])
rail_go_total = rail_go_total[["geo", "TIME_PERIOD", "OBS_VALUE"]]
rail_go_total = clean_geo(rail_go_total)
rail_go_total.rename(columns={"OBS_VALUE": "rail_f_total_Nat&INT_mio_tkm"}, inplace=True)

# --- Rail infrastructure length (robust selection) ---
rail_if_line_na = read_eurostat(files["rail_if_line_na"])

# Keep only total/main infrastructure and total/freight-only usage
rail_if_line_na = rail_if_line_na[
    rail_if_line_na["tra_infr"].isin(["TOTAL", "MAIN"]) &
    rail_if_line_na["tra_meas"].isin(["TOTAL", "FR_ONL"])
]

# Sort so TOTAL comes first → FR_ONL used only if TOTAL missing
rail_if_line_na.sort_values(
    by=["geo", "TIME_PERIOD", "tra_meas"],
    ascending=[True, True, False],  # TOTAL > FR_ONL
    inplace=True
)

# Drop duplicates, keeping TOTAL if available
rail_if_line_na = rail_if_line_na.drop_duplicates(
    subset=["geo", "TIME_PERIOD"],
    keep="first"
)

# Select and clean up
rail_if_line_na = rail_if_line_na[["geo", "TIME_PERIOD", "OBS_VALUE"]]
rail_if_line_na.rename(columns={"OBS_VALUE": "rail_length_km"}, inplace=True)
rail_if_line_na = clean_geo(rail_if_line_na)

# Modal split (rail share)------------------------------------------------------------
tran_hv_frmod = read_eurostat(files["tran_hv_frmod"])
tran_hv_frmod = tran_hv_frmod[tran_hv_frmod["tra_mode"] == "RAIL"]
tran_hv_frmod = tran_hv_frmod[["geo", "TIME_PERIOD", "OBS_VALUE"]]
tran_hv_frmod = clean_geo(tran_hv_frmod)
tran_hv_frmod.rename(columns={"OBS_VALUE": "rail_modal_share_pc"}, inplace=True)

# Consignment types--------------------------------------------------------------------
rail_go_consgmt = read_eurostat(files["rail_go_consgmt"])
rail_go_consgmt = rail_go_consgmt[["geo", "TIME_PERIOD", "consign", "OBS_VALUE"]]
rail_go_consgmt = clean_geo(rail_go_consgmt)
# Pivot to columns per consignment type
rail_go_consgmt = rail_go_consgmt.pivot_table(
    index=["geo", "TIME_PERIOD"], columns="consign", values="OBS_VALUE"
).reset_index()
rail_go_consgmt.columns.name = None
rail_go_consgmt.rename(columns={
    "FT": "full_train_tkm",
    "FW": "full_wagon_tkm",
    "OT": "other_tkm",
    "TOTAL": "total_tkm"
}, inplace=True)

# --- Commodity groups (NST 2007 classification) --------------------------------
rail_go_grpgood = read_eurostat(files["rail_go_grpgood"])

# Keep relevant columns
rail_go_grpgood = rail_go_grpgood[["geo", "TIME_PERIOD", "nst07", "unit", "OBS_VALUE"]]

# Filter to keep consistent unit (million tonne-kilometres)
rail_go_grpgood = rail_go_grpgood[rail_go_grpgood["unit"] == "MIO_TKM"].copy()
rail_go_grpgood.drop(columns=["unit"], inplace=True)

# Remove TOTAL rows (we'll compute totals manually later if needed)
rail_go_grpgood = rail_go_grpgood[rail_go_grpgood["nst07"] != "TOTAL"]

# Pivot: each NST07 commodity group becomes its own column
rail_go_grpgood = (
    rail_go_grpgood.pivot_table(
        index=["geo", "TIME_PERIOD"],
        columns="nst07",
        values="OBS_VALUE",
        aggfunc="sum"
    )
    .reset_index()
)
rail_go_grpgood.columns.name = None  # remove pivot metadata

# Rename GTxx → NSTxx for clarity and match analysis schema
rail_go_grpgood.rename(columns={
    "GT01": "NST01_Agriculture",
    "GT02": "NST02_Mining",
    "GT03": "NST03_MetalOres",
    "GT04": "NST04_FoodProducts",
    "GT05": "NST05_Textiles",
    "GT06": "NST06_Wood",
    "GT07": "NST07_Coke_Petroleum",
    "GT08": "NST08_Chemicals",
    "GT09": "NST09_NonMetallicMinerals",
    "GT10": "NST10_Metals",
    "GT11": "NST11_Machinery",
    "GT12": "NST12_TransportEquipment",
    "GT13": "NST13_Furniture",
    "GT14": "NST14_SecondaryRawMaterials",
    "GT15": "NST15_Waste",
    "GT16": "NST16_Mail",
    "GT17": "NST17_Equipment",
    "GT18": "NST18_HouseholdGoods",
    "GT19": "NST19_EmptyContainers",
    "GT20": "NST20_OtherGoods"
}, inplace=True)

# Clean country codes (ensure consistent GEO identifiers)
rail_go_grpgood = clean_geo(rail_go_grpgood)

Loading: rail_go_total__custom_18309054_linear_2_0.csv
Loading: rail_if_line_na__custom_18324484_linear_2_0.csv
Loading: tran_hv_frmod__custom_18309026_linear_2_0.csv
Loading: rail_go_consgmt__custom_18308929_linear_2_0.csv
Loading: rail_go_grpgood__custom_18309135_linear_2_0.csv


Merge datasets

In [4]:
df = (
    rail_go_total
    .merge(rail_if_line_na, on=["geo", "TIME_PERIOD"], how="left")
    .merge(tran_hv_frmod, on=["geo", "TIME_PERIOD"], how="left")
    .merge(rail_go_consgmt, on=["geo", "TIME_PERIOD"], how="left")
    .merge(rail_go_grpgood, on=["geo", "TIME_PERIOD"], how="left")
)

print("Merged dataset shape:", df.shape)
print("NST columns found:", [c for c in df.columns if c.startswith("NST")])
##Build path
DIR = os.path.expanduser(r"~\OneDrive\Desktop\TIL Programming\6020 Group project\Project data_Freight")
merged_path = os.path.join(DIR, "merged_eurostat_clean.csv")

# Only now save
df.to_csv(merged_path, index=False)
print("Saved updated merged dataset to disk.")

Merged dataset shape: (848, 28)
NST columns found: ['NST01_Agriculture', 'NST02_Mining', 'NST03_MetalOres', 'NST04_FoodProducts', 'NST05_Textiles', 'NST06_Wood', 'NST07_Coke_Petroleum', 'NST08_Chemicals', 'NST09_NonMetallicMinerals', 'NST10_Metals', 'NST11_Machinery', 'NST12_TransportEquipment', 'NST13_Furniture', 'NST14_SecondaryRawMaterials', 'NST15_Waste', 'NST16_Mail', 'NST17_Equipment', 'NST18_HouseholdGoods', 'NST19_EmptyContainers', 'NST20_OtherGoods']
Saved updated merged dataset to disk.


Quick Check:

In [5]:
# Path to your merged dataset
DIR = os.path.expanduser(r"~\OneDrive\Desktop\TIL Programming\6020 Group project\Project data_Freight")
merged_path = os.path.join(DIR, "merged_eurostat_clean.csv")

# Load into pandas
df = pd.read_csv(merged_path)

# Quick preview
df.head()

,geo,TIME_PERIOD,rail_f_total_Nat&INT_mio_tkm,rail_length_km,rail_modal_share_pc,total_tkm,TRN_FUL,WAG_FUL,NST01_Agriculture,NST02_Mining,...,NST11_Machinery,NST12_TransportEquipment,NST13_Furniture,NST14_SecondaryRawMaterials,NST15_Waste,NST16_Mail,NST17_Equipment,NST18_HouseholdGoods,NST19_EmptyContainers,NST20_OtherGoods
0,AT,2008,21915.0,5693.0,33.6,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AT,2009,17767.0,5693.0,32.0,NaN,NaN,NaN,1890.0,740.0,...,81.0,980.0,4.0,839.0,11.0,239.0,4.0,23.0,3918.0,0.0
2,AT,2010,19833.0,5828.0,33.0,NaN,NaN,NaN,2039.0,681.0,...,68.0,1042.0,5.0,1162.0,13.0,420.0,127.0,255.0,4649.0,0.0
3,AT,2011,20345.0,5500.0,33.1,NaN,NaN,NaN,2017.0,733.0,...,71.0,1080.0,8.0,1242.0,14.0,419.0,167.0,330.0,4607.0,0.0
4,AT,2012,19499.0,5566.0,32.7,NaN,NaN,NaN,1786.0,712.0,...,63.0,1031.0,7.0,1102.0,11.0,408.0,188.0,370.0,4079.0,0.0


Check structure:

In [6]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   geo                           848 non-null    object 
 1   TIME_PERIOD                   848 non-null    int64  
 2   rail_f_total_Nat&INT_mio_tkm  812 non-null    float64
 3   rail_length_km                798 non-null    float64
 4   rail_modal_share_pc           798 non-null    float64
 5   total_tkm                     330 non-null    float64
 6   TRN_FUL                       348 non-null    float64
 7   WAG_FUL                       324 non-null    float64
 8   NST01_Agriculture             836 non-null    float64
 9   NST02_Mining                  834 non-null    float64
 10  NST03_MetalOres               832 non-null    float64
 11  NST04_FoodProducts            834 non-null    float64
 12  NST05_Textiles                830 non-null    float64
 13  NST06

,geo,TIME_PERIOD,rail_f_total_Nat&INT_mio_tkm,rail_length_km,rail_modal_share_pc,total_tkm,TRN_FUL,WAG_FUL,NST01_Agriculture,NST02_Mining,...,NST11_Machinery,NST12_TransportEquipment,NST13_Furniture,NST14_SecondaryRawMaterials,NST15_Waste,NST16_Mail,NST17_Equipment,NST18_HouseholdGoods,NST19_EmptyContainers,NST20_OtherGoods
count,848,848.000000,812.000000,798.000000,798.000000,330.000000,348.000000,324.000000,836.000000,834.000000,...,836.000000,832.000000,836.000000,832.000000,810.000000,834.000000,830.000000,832.000000,836.000000,824.000000
unique,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2016.009434,39320.791872,8117.365569,23.938095,46591.000000,29333.709770,18993.753086,938.114833,1544.136691,...,38.648325,461.807692,13.892344,340.271635,6.412346,208.505995,18.691566,308.882212,3576.392344,219.070388
std,NaN,4.903788,62711.790161,9285.582601,18.613001,60216.420849,47959.991855,18138.013539,1037.710786,3240.047867,...,79.733408,1060.326972,54.813875,594.241079,27.768696,536.875452,104.164625,829.262383,8319.114614,497.345549
min,NaN,2008.000000,67.000000,271.000000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2012.000000,6532.000000,2240.000000,10.800000,10377.500000,0.000000,3157.500000,43.000000,6.000000,...,2.000000,11.750000,0.000000,18.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000
50%,NaN,2016.000000,16245.500000,3627.000000,22.100000,27959.000000,7341.500000,20287.500000,517.000000,330.000000,...,13.000000,92.500000,1.000000,102.500000,0.000000,43.000000,0.000000,5.000000,731.500000,0.000000
75%,NaN,2020.000000,48745.500000,10777.000000,30.650000,44278.000000,28374.500000,29952.000000,1547.000000,1021.000000,...,37.000000,455.750000,5.000000,392.500000,0.000000,162.000000,1.000000,105.500000,3327.000000,148.500000


Check for missing data

In [7]:
(df.notna().sum() / len(df) * 100).round(1)

geo                             100.0
TIME_PERIOD                     100.0
rail_f_total_Nat&INT_mio_tkm     95.8
rail_length_km                   94.1
rail_modal_share_pc              94.1
total_tkm                        38.9
TRN_FUL                          41.0
WAG_FUL                          38.2
NST01_Agriculture                98.6
NST02_Mining                     98.3
NST03_MetalOres                  98.1
NST04_FoodProducts               98.3
NST05_Textiles                   97.9
NST06_Wood                       98.6
NST07_Coke_Petroleum             97.6
NST08_Chemicals                  98.6
NST09_NonMetallicMinerals        98.6
NST10_Metals                     98.6
NST11_Machinery                  98.6
NST12_TransportEquipment         98.1
NST13_Furniture                  98.6
NST14_SecondaryRawMaterials      98.1
NST15_Waste                      95.5
NST16_Mail                       98.3
NST17_Equipment                  97.9
NST18_HouseholdGoods             98.1
NST19_EmptyC